In [ ]:
print("pierwsze polecenie uruchomi context sparka")

In [ ]:
dir(spark)

## Pan Tadeusz

In [ ]:
## ładujemy zbiór z S3
pan_tadeusz = spark.read.text("s3://kompleksowe-szkolenie-bigdata/warsztat_02/pan_tadeusz.txt")

In [ ]:
## tworzymy tabelę tymczasową
pan_tadeusz.createOrReplaceTempView('pan_tadeusz_tabelka')

In [ ]:
spark.sql("""select * from pan_tadeusz_tabelka""").show(20)

In [ ]:
spark.sql("""select count(*) from pan_tadeusz_tabelka""").show()

In [ ]:
pan_tadeusz.count()

In [ ]:
pan_tadeusz_wyrazy = spark.sql("""select explode(split(value,' ')) from pan_tadeusz_tabelka""")

In [ ]:
pan_tadeusz_wyrazy.show(10)

In [ ]:
pan_tadeusz_wyrazy.distinct().count()

In [ ]:
spark.sql("""select count(distinct wyrazy) from (select explode(split(value,' ')) as wyrazy from pan_tadeusz_tabelka)""").show()

In [ ]:
pan_tadeusz.write.text("hdfs:///tmp/pan_tadeusz3")

## Users

In [ ]:
users = spark.read.parquet('s3://kompleksowe-szkolenie-bigdata/stackoverflow/users_pq')

In [ ]:
spark.read.parquet('s3://kompleksowe-szkolenie-bigdata/stackoverflow/users_pq').write.parquet("hdfs:///tmp/users_parquet")

In [ ]:
users = spark.read.parquet("hdfs:///tmp/users_parquet")

In [ ]:
users.createOrReplaceTempView('users_tabelka')

In [ ]:
users.printSchema()

In [ ]:
users.describe().show()

In [ ]:
users.take(2)

In [ ]:
users.select('id','displayname').take(5)

In [ ]:
users.select('id','displayname').show(5)

In [ ]:
cache_example=users.select('id','displayname','views').cache()

In [ ]:
cache_example.show(5)

In [ ]:
cache_example.show(5)

###  Z jakiego państwa jest najwięcej użytkowników ?

In [ ]:
users.groupBy("location").agg({"location": "count"}).show()

In [ ]:
users.groupBy("location").agg({"location": "count"}).orderBy('count(location)',ascending=0).show()

### Ile jest unikalnych lokalizacji ?

In [ ]:
spark.sql("""select count(distinct location) from users_tabelka""").show()

In [ ]:
users.select('location').dropDuplicates().count()

In [ ]:
spark.sql("""select count(*) from users_tabelka where location != "" """).show()

### jak można filtrować dane?

In [ ]:
spark.sql("""select displayname,upvotes from users_tabelka where upvotes > 100""").show()

In [ ]:
users.select('displayname','upvotes').filter(users.upvotes > 100).show()

### agregacje

In [ ]:
users.groupBy('reputation').agg({'upvotes':'mean','downvotes':'mean','reputation':'count'}).show()

### sortowanie

In [ ]:
users.orderBy(users.reputation.desc()).select('displayname',users.reputation).show(20)

In [ ]:
users.orderBy(users.reputation, ascending=0).select(users.displayname,users.reputation).show()

### jak dodac nowa kolumnę ?

In [ ]:
users.withColumn('reputation_calculation', users.reputation *2).printSchema()

In [ ]:
users.withColumn('reputation_calculation', users.reputation *2). \
    select('displayname','reputation','upvotes','reputation_calculation'). \
    show(10)

### jak usunąć kolumnę ?

In [ ]:
users.drop('views').printSchema()